In [26]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
import time
import csv
import re
from geopy.geocoders import Nominatim
import folium

In [29]:
# Define a regular expression pattern for IDs that are mixed with numbers and alphabet
pattern = re.compile(r'[0-9]+[a-zA-Z]+|[a-zA-Z]+[0-9]+')

with open('FilteredData.csv', 'r') as file:
    reader = csv.DictReader(file)

    # Create a list of the values that are mixed with numbers and alphabet
    mixed_ids = [row['ID'] for row in reader if pattern.match(row['ID'])]

print(mixed_ids)
len(mixed_ids)

['15A', '15D', '5A3', '5A4', '5A5', '5AT', '5C1', '5C4', '5C5', '5C9', '5CN', '5CQ', '5EF', '5F1', '5FE', '5FL', '5H8', '5J2', '5J9', '5JE', '5K3', '5K5', '5K7', '5KG', '5KL', '5L1', '5L3', '5LA', '5LG', '5M1', '5M2', '5M3', '5M6', '5M7', '5M8', '5MD', '5MK', '5MV', '5MX', '5N2', '5N5', '5N6', '5N8', '5N9', '5NE', '5NG', '5NH', '5NL', '5NN', '5NV', '5NW', '5P3', '5P4', '5P5', '5P6', '5P7', '5P8', '5PA', '5PC', '5PD', '5PE', '5PG', '5PH', '5PJ', '5PK', '5PL', '5PM', '5PP', '5PQ', '5PR', '5PT', '5PV', '5PX', '5PY', '5QA', '5QC', '5QD', '5QE', '5QF', '5QG', '5QH', '5QK', '5QL', '5QM', '5QN', '5QP', '5QQ', '5QR', '5QT', '8A101', '8A718', '8E339', '8G301', '8HE28', '8HP32', '8J285', 'A0C5S', 'A4M8P', 'AAH01', 'AAV02', 'ACG05', 'ACG06', 'ACG07', 'ACG08', 'ACG09', 'ACG10', 'ACG12', 'ACG13', 'ACG19', 'ACG20', 'ACG22', 'ACG24', 'ACG30', 'ACG36', 'ACG37', 'ACG38', 'ACG40', 'ACG41', 'ADP02', 'AHH01', 'AHR01', 'AJX01', 'ARH01', 'AVQ01', 'AVQ03', 'AXG01', 'AXG02', 'AXG03', 'B3M1X', 'B4N1U', 'B5M7B'

742

In [32]:
# Open the CSV file
with open('FilteredData.csv', 'r') as file:
    reader = csv.DictReader(file)

    # Create a list of all the IDs
    all_ids = [row['ID'] for row in reader]

# Create a list of the IDs that are not in mixed_ids
not_mixed_ids = [id for id in all_ids if id not in mixed_ids]

print(not_mixed_ids)
len(not_mixed_ids)

['England', 'ENGLAND', 'NTPAD', 'NTPAE', 'NTPAT', 'RAE', 'RAJ', 'RAL', 'RAN', 'RAP', 'RAS', 'RAT', 'RAX', 'RBA', 'RBB', 'RBD', 'RBF', 'RBF-X', 'RBK', 'RBL', 'RBN', 'RBQ', 'RBS', 'RBT', 'RBV', 'RBZ', 'RCB', 'RCC', 'RCD', 'RCF', 'RCU', 'RCX', 'RDD', 'RDE', 'RDR', 'RDU', 'RDY', 'RDZ', 'REF', 'REF-X', 'REM', 'REN', 'REP', 'RET', 'RFF', 'RFR', 'RFS', 'RFW', 'RGC', 'RGD', 'RGM', 'RGN', 'RGP', 'RGQ', 'RGR', 'RGT', 'RGZ', 'RHA', 'RHM', 'RHQ', 'RHU', 'RHW', 'RHX', 'RJC', 'RJD', 'RJD-X', 'RJDTC', 'RJE', 'RJF', 'RJL', 'RJL-X', 'RJN', 'RJR', 'RJX', 'RJZ', 'RKB', 'RKE', 'RKL', 'RLN', 'RLQ', 'RLT', 'RLU', 'RLY', 'RMC', 'RMP', 'RMY', 'RMY-X', 'RNA', 'RNH', 'RNJ', 'RNL', 'RNN', 'RNQ', 'RNS', 'RNU', 'RNZ', 'RPA', 'RPC', 'RPG', 'RPL', 'RPR', 'RPY', 'RQM', 'RQQ', 'RQQ-X', 'RQW', 'RQX', 'RQY', 'RRD', 'RRE', 'RRF', 'RRJ', 'RRK', 'RRK-X', 'RRP', 'RRV', 'RTD', 'RTE', 'RTF', 'RTG', 'RTH', 'RTK', 'RTP', 'RTQ', 'RTR', 'RTV', 'RTX', 'RVJ', 'RVJ-X', 'RVL', 'RVN', 'RVR', 'RVR-X', 'RVV', 'RVW', 'RVY', 'RWA', 'RWD',

190

In [34]:
def scrape_data(base_url, codes):
    # new Chrome browser instance
    webdriver_service = Service("C:\Program Files (x86)\chromedriver.exe")
    driver = webdriver.Chrome(service=webdriver_service)

    # Open the CSV file in write mode
    with open('scraped_data.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        # Write the header row
        writer.writerow(['ID', 'Name', 'National Grouping', 'Higher Health Authority', 'Opened', 'Closed', 'Address'])

        for code in codes:
            try:
                # Navigate to the base page
                driver.get(base_url)

                # Wait for the page to load
                time.sleep(1)

                # Find the input box and enter the code
                input_box = driver.find_element('id', 'Code')
                input_box.send_keys(code)

                # Find the search button and click it
                search_button = driver.find_element('id', 'Search')
                search_button.click()

                # Wait for the new page to load
                time.sleep(1)

                # Get the page source
                page_source = driver.page_source

                # Create a BeautifulSoup object and specify the parser
                soup = BeautifulSoup(page_source, 'html.parser')

                # Find the name
                name_element = soup.find('div', attrs={'class': 'row col-lg-12'})
                if name_element is not None:
                    name = name_element.text.strip()
                else:
                    name = "N/A"

                # Find the data 
                data = soup.find_all('div', attrs={'class': 'row'})

                # Extract the information from the data
                row_data = {'ID': code, 'Name': name}
                for element in data:
                    label = element.find('div', attrs={'class': 'col-lg-3 control-label'})
                    value = element.find('div', attrs={'class': 'col-lg-9'})
                    if label and value:
                        label_text = label.text.strip().replace(':', '')
                        value_text = ' '.join(value.text.split())
                        row_data[label_text] = value_text

                # Ensure all fields are present in the row
                for field in ['ID', 'Name', 'National Grouping', 'Higher Health Authority', 'Opened', 'Closed', 'Address']:
                    if field not in row_data:
                        row_data[field] = 'N/A'

                # Write the row data in the correct order
                writer.writerow([row_data[field] for field in ['ID', 'Name', 'National Grouping', 'Higher Health Authority', 'Opened', 'Closed', 'Address']])
            except NoSuchElementException:
                print(f"Unable to find element for code: {code}. Skipping to next code.")
                continue

    driver.quit()

base_url = 'https://odsportal.digital.nhs.uk/Organisation/Search/'

codes = mixed_ids

scrape_data(base_url, codes)


In [30]:
# Create a map centered around a general location
m = folium.Map(location=[51.5074, 0.1278], zoom_start=6)

# Create a geolocator object
geolocator = Nominatim(user_agent="measurements", timeout=3)

# Open the CSV file
with open('scraped_data.csv', 'r') as file:
    reader = csv.DictReader(file)

    # Counter for the number of addresses processed
    count = 0

    # For each row in the CSV...
    for row in reader:
        # If we've already processed 5 addresses, break the loop
        if count >= 100:
            break

        # Get the address
        address = row['Address']

        # Try to geolocate the address
        location = geolocator.geocode(address)

        # If the address was geolocated successfully...
        if location:
            # Add a marker to the map
            folium.Marker([location.latitude, location.longitude]).add_to(m)

            # Increment the counter
            count += 1

# Save the map to an HTML file
m.save('map.html')
m